In [4]:
#https://github.com/vrasneur/udacity-deep_learning/blob/master/4_convolutions.ipynb
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [5]:
# Load train dataset
with open('train_dataset_Colors.pickle', 'rb') as handle:
    X_train = pickle.load(handle)
with open('train_labels_Colors.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
    y_train = y_train.ravel()

# Load test dataset
with open('test_dataset_Colors.pickle', 'rb') as handle:
    X_test = pickle.load(handle)
with open('test_labels_Colors.pickle', 'rb') as handle:
    y_test = pickle.load(handle)
    y_test = y_test.ravel()

# Load validation dataset
with open('valid_dataset_Colors.pickle', 'rb') as handle:
    X_val = pickle.load(handle)
with open('valid_labels_Colors.pickle', 'rb') as handle:
    y_val = pickle.load(handle)
    y_val = y_val.ravel()


print('Training set', X_train.shape, y_train.shape)
print('Validation set', X_val.shape, y_val.shape)
print('Test set', X_test.shape, y_test.shape)


Training set (1481, 60, 60, 3) (1481,)
Validation set (318, 60, 60, 3) (318,)
Test set (317, 60, 60, 3) (317,)


In [6]:
image_size = 60
num_labels = 2
num_channels = 3 # RGB

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels


train_dataset, train_labels = reformat(X_train, y_train)
valid_dataset, valid_labels = reformat(X_val, y_val)
test_dataset, test_labels = reformat(X_test, y_test)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (1481, 60, 60, 3) (1481, 2)
Validation set (318, 60, 60, 3) (318, 2)
Test set (317, 60, 60, 3) (317, 2)


In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [8]:
print('change the optimizer (use Adam)')
print('increase batch size')
print('increase the number of steps')
print('increase the number of hidden nodes')

change the optimizer (use Adam)
increase batch size
increase the number of steps
increase the number of hidden nodes


In [27]:
batch_size = 200
patch_size = 5
depth = 16
num_hidden = 128

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data, train=False):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
        shape = hidden.get_shape().as_list()
        print(shape, train)
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset, True)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    optimizer = tf.train.AdamOptimizer().minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

[200, 15, 15, 16] True
[318, 15, 15, 16] False
[317, 15, 15, 16] False


In [28]:
num_steps = 2001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        #hristo batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_data = train_dataset[offset:(offset + batch_size),:, :, :]

        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1.050052
Minibatch accuracy: 68.0%
Validation accuracy: 33.3%
Minibatch loss at step 50: 0.377788
Minibatch accuracy: 84.5%
Validation accuracy: 84.6%
Minibatch loss at step 100: 0.183252
Minibatch accuracy: 96.0%
Validation accuracy: 92.1%
Minibatch loss at step 150: 0.093429
Minibatch accuracy: 97.0%
Validation accuracy: 94.7%
Minibatch loss at step 200: 0.032489
Minibatch accuracy: 100.0%
Validation accuracy: 96.2%
Minibatch loss at step 250: 0.022125
Minibatch accuracy: 100.0%
Validation accuracy: 97.8%
Minibatch loss at step 300: 0.023475
Minibatch accuracy: 99.5%
Validation accuracy: 97.8%
Minibatch loss at step 350: 0.015858
Minibatch accuracy: 99.5%
Validation accuracy: 97.5%
Minibatch loss at step 400: 0.011011
Minibatch accuracy: 100.0%
Validation accuracy: 98.1%
Minibatch loss at step 450: 0.004783
Minibatch accuracy: 100.0%
Validation accuracy: 97.5%
Minibatch loss at step 500: 0.003853
Minibatch accuracy: 100.0%
Validation accuracy: 97

In [22]:
print('dropout')
print('L2 regularization')
print('huge number of steps')

(317, 1)
(317,)


In [30]:
import operator

batch_size = 200
patch_size = 5
depth = 16
num_hidden = 128

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data, train=False):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
        if train:
            hidden = tf.nn.dropout(hidden, 0.5)
        shape = hidden.get_shape().as_list()
        print(shape, train)
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        if train:
            hidden = tf.nn.dropout(hidden, 0.5)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset, True)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    loss += 5e-4 * reduce(operator.add, [tf.nn.l2_loss(w) for w in (layer1_weights, layer2_weights,
                                                              layer3_weights, layer4_weights)])

    optimizer = tf.train.AdamOptimizer().minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

[200, 15, 15, 16] True
[318, 15, 15, 16] False
[317, 15, 15, 16] False


In [31]:
num_steps = 2001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        #hristo batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        #last change batch_data = train_dataset[offset:(offset + batch_size),:, :, :]
        batch_data = train_dataset[offset:(offset + batch_size),...] #changed to ...

        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.643738
Minibatch accuracy: 44.5%
Validation accuracy: 66.7%
Minibatch loss at step 50: 1.480675
Minibatch accuracy: 71.0%
Validation accuracy: 66.7%
Minibatch loss at step 100: 1.384079
Minibatch accuracy: 68.0%
Validation accuracy: 75.8%
Minibatch loss at step 150: 1.224671
Minibatch accuracy: 77.0%
Validation accuracy: 85.5%
Minibatch loss at step 200: 0.994580
Minibatch accuracy: 91.5%
Validation accuracy: 91.8%
Minibatch loss at step 250: 0.806820
Minibatch accuracy: 97.5%
Validation accuracy: 95.6%
Minibatch loss at step 300: 0.830300
Minibatch accuracy: 96.5%
Validation accuracy: 97.8%
Minibatch loss at step 350: 0.750863
Minibatch accuracy: 97.5%
Validation accuracy: 98.1%
Minibatch loss at step 400: 0.695802
Minibatch accuracy: 98.5%
Validation accuracy: 98.1%
Minibatch loss at step 450: 0.653468
Minibatch accuracy: 100.0%
Validation accuracy: 97.8%
Minibatch loss at step 500: 0.640215
Minibatch accuracy: 98.5%
Validation accuracy: 97.8%


In [1]:
print("remove dropout")
print("remove L2 regularization")

remove dropout
remove L2 regularization


In [9]:
import operator

batch_size = 200
patch_size = 5
depth = 16
num_hidden = 128

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data, train=False):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
        shape = hidden.get_shape().as_list()
        print(shape, train)
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset, True)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    optimizer = tf.train.AdamOptimizer().minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

[200, 15, 15, 16] True
[318, 15, 15, 16] False
[317, 15, 15, 16] False


In [11]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 0.652196
Minibatch accuracy: 68.0%
Validation accuracy: 33.3%
Minibatch loss at step 50: 0.344735
Minibatch accuracy: 87.5%
Validation accuracy: 86.5%
Minibatch loss at step 100: 0.096671
Minibatch accuracy: 96.0%
Validation accuracy: 94.0%
Minibatch loss at step 150: 0.024897
Minibatch accuracy: 99.5%
Validation accuracy: 97.5%
Minibatch loss at step 200: 0.008787
Minibatch accuracy: 100.0%
Validation accuracy: 98.1%
Minibatch loss at step 250: 0.007405
Minibatch accuracy: 100.0%
Validation accuracy: 98.7%
Minibatch loss at step 300: 0.008111
Minibatch accuracy: 100.0%
Validation accuracy: 98.4%
Minibatch loss at step 350: 0.004872
Minibatch accuracy: 100.0%
Validation accuracy: 98.4%
Minibatch loss at step 400: 0.002060
Minibatch accuracy: 100.0%
Validation accuracy: 98.7%
Minibatch loss at step 450: 0.002460
Minibatch accuracy: 100.0%
Validation accuracy: 97.8%
Minibatch loss at step 500: 0.000882
Minibatch accuracy: 100.0%
Validation accuracy: 

In [12]:
print("Add L2 regularization again")

Add L2 regularization again


In [13]:
import operator

batch_size = 200
patch_size = 5
depth = 16
num_hidden = 128

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data, train=False):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
        shape = hidden.get_shape().as_list()
        print(shape, train)
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset, True)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    loss += 5e-4 * reduce(operator.add, [tf.nn.l2_loss(w) for w in (layer1_weights, layer2_weights,
                                                                  layer3_weights, layer4_weights)])

    optimizer = tf.train.AdamOptimizer().minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

[200, 15, 15, 16] True
[318, 15, 15, 16] False
[317, 15, 15, 16] False


In [14]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.450628
Minibatch accuracy: 68.0%
Validation accuracy: 33.3%
Minibatch loss at step 50: 1.167586
Minibatch accuracy: 82.5%
Validation accuracy: 81.1%
Minibatch loss at step 100: 0.875997
Minibatch accuracy: 94.5%
Validation accuracy: 93.7%
Minibatch loss at step 150: 0.727306
Minibatch accuracy: 99.0%
Validation accuracy: 95.0%
Minibatch loss at step 200: 0.672460
Minibatch accuracy: 99.5%
Validation accuracy: 96.2%
Minibatch loss at step 250: 0.639933
Minibatch accuracy: 100.0%
Validation accuracy: 97.2%
Minibatch loss at step 300: 0.620652
Minibatch accuracy: 100.0%
Validation accuracy: 97.5%
Minibatch loss at step 350: 0.601353
Minibatch accuracy: 99.5%
Validation accuracy: 98.4%
Minibatch loss at step 400: 0.584356
Minibatch accuracy: 100.0%
Validation accuracy: 97.5%
Minibatch loss at step 450: 0.571626
Minibatch accuracy: 100.0%
Validation accuracy: 97.8%
Minibatch loss at step 500: 0.552566
Minibatch accuracy: 100.0%
Validation accuracy: 97

In [15]:
print("Add just dropout (no L2 reg)")

Add just dropout (no L2 reg)


In [16]:
import operator

batch_size = 200
patch_size = 5
depth = 16
num_hidden = 128

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data, train=False):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
        if train:
            hidden = tf.nn.dropout(hidden, 0.5)
        shape = hidden.get_shape().as_list()
        print(shape, train)
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        if train:
            hidden = tf.nn.dropout(hidden, 0.5)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset, True)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    optimizer = tf.train.AdamOptimizer().minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

[200, 15, 15, 16] True
[318, 15, 15, 16] False
[317, 15, 15, 16] False


In [17]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.850452
Minibatch accuracy: 47.0%
Validation accuracy: 66.7%
Minibatch loss at step 50: 0.558823
Minibatch accuracy: 69.5%
Validation accuracy: 73.9%
Minibatch loss at step 100: 0.447896
Minibatch accuracy: 75.0%
Validation accuracy: 81.8%
Minibatch loss at step 150: 0.285867
Minibatch accuracy: 86.5%
Validation accuracy: 90.6%
Minibatch loss at step 200: 0.125059
Minibatch accuracy: 97.5%
Validation accuracy: 94.7%
Minibatch loss at step 250: 0.037916
Minibatch accuracy: 99.0%
Validation accuracy: 96.5%
Minibatch loss at step 300: 0.090352
Minibatch accuracy: 97.0%
Validation accuracy: 97.2%
Minibatch loss at step 350: 0.053891
Minibatch accuracy: 99.0%
Validation accuracy: 97.8%
Minibatch loss at step 400: 0.040579
Minibatch accuracy: 98.5%
Validation accuracy: 97.8%
Minibatch loss at step 450: 0.021143
Minibatch accuracy: 99.0%
Validation accuracy: 97.8%
Minibatch loss at step 500: 0.002321
Minibatch accuracy: 100.0%
Validation accuracy: 98.1%
